<a href="https://colab.research.google.com/github/aml-2023/final-project/blob/yolo/yolo_ultralytics" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install
We will install the `ultralytics` pip package which lets us easily fine tune YOLO models.

https://github.com/ultralytics/ultralytics

In [1]:
!pip install ultralytics -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.1/660.1 kB 9.0 MB/s eta 0:00:00


# Fetch Data
Run the `fetch_data` to get the garbage data in the yolo format.

In [2]:
!wget -O fetch_data.sh https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
!bash fetch_data.sh --type yolo --output garbage

--2023-12-08 08:32:20--  https://raw.githubusercontent.com/aml-2023/final-project/main/fetch_data.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1222 (1.2K) [text/plain]
Saving to: ‘fetch_data.sh’

fetch_data.sh       100%[===================>]   1.19K  --.-KB/s    in 0s      

2023-12-08 08:32:20 (8.22 MB/s) - ‘fetch_data.sh’ saved [1222/1222]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0   1095      0 --:--:-- --:--:-- --:--:--  1095
100  274M  100  274M    0     0  96.6M      0  0:00:02  0:00:02 --:--:--  170M
Data downloaded and extracted into garbage


# Create data subset
We don't want to train on the entire dataset because it takes too long, so let's just look at a subset. The functions below copy over p% of files into a new directory.

In [3]:
import os
import shutil
from glob import glob
import numpy as np
import pathlib

def label_path_from_image_path(image_path: str, base_path):
    """Gets the YOLO label path from the image path."""
    label = image_path.split("/")[-1]
    label = label[:-3] + "txt"
    label = os.path.join(base_path, label)
    return label


def get_image_label_path_pair(base_path: str, split_folder: str):
    """Gets all the image and label path pairs for a specific base path and the split folder, e.g. garbage and test."""
    img_path = os.path.join(base_path, split_folder, "images")
    labels_path = os.path.join(base_path, split_folder, "labels")

    pairs = []
    for image_name in glob(f"{img_path}/*.jpg"):
        label = label_path_from_image_path(image_name, labels_path)
        pairs.append((image_name, label))

    return pairs


def subset_split_folder(yolo_root_dir: str, percentage: float, split_folder: str, out_dir: str):
    """Subsets a split folder (train, test, valid) and copies the subset to a new directory."""
    pairs = get_image_label_path_pair(yolo_root_dir, split_folder)
    subset_len = int(len(pairs) * percentage)
    subset_idx = np.random.randint(low=0, high=len(pairs), size=subset_len)

    subset_pairs = [pairs[i] for i in subset_idx]

    out_dir_img = os.path.join(out_dir, split_folder, "images")
    out_dir_labels = os.path.join(out_dir, split_folder, "labels")

    pathlib.Path(out_dir_img).mkdir(parents=True, exist_ok=True)
    pathlib.Path(out_dir_labels).mkdir(parents=True, exist_ok=True)

    for img, label in subset_pairs:
        dest_img_path = os.path.join(out_dir_img, img.split("/")[-1])
        dest_label_path = os.path.join(out_dir_labels, label.split("/")[-1])

        shutil.copy(img, dest_img_path)
        shutil.copy(label, dest_label_path)

def subset_yolo_data(yolo_root_dir: str, percentage: float, out_dir: str):
    """Subsets the YOLO dataset by taking a percentage of the original data and moving it into a new directory.

    :arg
        yolo_root_dir (str): the root directory where the yolo data is.
        percentage (float): the percentage of images to keep.
        out_dir (str): the output directory, will be created if it does not exist.
    """
    subset_split_folder(yolo_root_dir, percentage, "train", out_dir)
    subset_split_folder(yolo_root_dir, percentage, "test", out_dir)
    subset_split_folder(yolo_root_dir, percentage, "valid", out_dir)

    other_files = ["README.dataset.txt", "README.roboflow.txt", "data.yaml"]

    for file in other_files:
        old_path = os.path.join(yolo_root_dir, file)
        new_path = os.path.join(out_dir, file)
        shutil.copy(old_path, new_path)


Let's only use 10 percent of the data

In [4]:
subset_yolo_data("garbage", 0.1, "garbage_sub")

# Training the Model
Let's train the model! We just need to import the `YOLO` class from the `ultralytics` package, tell it which version we are using (we go for the nano model because it is fastest to train), and then train it for a number of epochs.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="/content/garbage_sub/data.yaml", epochs=25)  # train the model
metrics = model.val()  # evaluate model performance on the validation set